**Titanic Competition Introduction:**

Our problem is to predict which passengers survived the Titanic's maiden voyage given the following information about them:

- *Age* in years old
- *Sex* $\in$ {female, male}
- *SibSp* representing the number of siblings and spouses travelling with the passenger
- *Parch* representing the number of parents and children travelling with the passenger
- *Pclass* representing the passenger's ticket class $\in$ {1st class, 2nd class, 3rd class}
- *Ticket* number
- *Cabin* number
- *Fare* paid for ticket
- *Embarked* representing the port where the passenger embarked $\in$ {C = Cherbourg, Q = Queenstown, S = Southampton}
- *Survived* $\in$ {0 = died, 1 = survived} for the training set, target for test 

This represents a binary classification problem as the passengers either survived, or they did not.

In [ ]:
# Data analysis
import numpy as np
import pandas as pd
pd.set_option("display.precision", 4)
from math import isnan

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error

# Peace of mind
import warnings
warnings.filterwarnings('ignore')

# Define constants
TRAIN_LEN = 891
RNG_SEED = 343
COLS_TO_DROP = []
CHILD_AGE_END = 18
DECKS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'N']
DEFAULT_SURVIVAL = 0.5

##########################################
########## Function definitions ##########
##########################################
def get_deck_class_count(df, T_deck):
    """
    get_deck_class_count reformats counts of Titanic passengers for deck, class
    combinations into a dictionary along with providing the percentages/
    proportion of each class on each deck

    :param df: dataframe grouped by Deck and Pclass storing the count for 
        each combination
    :param T_deck: boolean that indicates if we are including 'T' deck or not
    :return: two dictionaries with passenger class counts and proportions for 
        each deck
    """ 
    # Dictionaries storing passenger counts and proportions for all deck, 
    # class combinations
    deck_count = {'A': {}, 'B': {}, 'C': {}, 'D': {}, 'E': {}, 'F': {},
                  'G': {}, 'N': {}, 'T': {}}
    if not T_deck:
        deck_count.pop('T', None)
    deck_percent = {}
    decks = df.transpose().columns.levels[0] # List of decks
    
    for deck in decks:
        # Populate deck_count
        for pclass in range(1, 4):
            try:
                count = int(df.loc[deck, pclass])
                deck_count[deck][pclass] = count
            except KeyError:
                deck_count[deck][pclass] = 0
                
        # Populate deck_percent
        deck_percent[deck] = [(count / sum(deck_count[deck].values())) 
                              * 100 for count in deck_count[deck].values()]
        
    return deck_count, deck_percent    


def get_surv_prop(deck, pclass):
    """
    surv_prop provides the percentage survival for passengers given their deck
    and pclass

    :param deck: string denoting the deck of a passenger
    :param pclass: integer denoting the pclass of a passenger
    :return: decimal percentage of survival for similar passengers
    """ 
    
    return deck_class_surv_prop[deck][pclass]


def get_corr(df):
    """
    get_corr calculates and returns a dataframe of correlations for an input
    dataframe's features
    
    :param df: dataframe containing features we want correlations of
    :return: dataframe of correlations excluding duplicate and self correlations
    """
    # Calculate correlation and sort output
    corr = df.corr().abs().unstack().sort_values(kind="quicksort", 
                                                 ascending=False).reset_index()
    
    # Remap correlation and drop duplicate/self correlations
    cols_map = {"level_0": "Feature 1", 
                "level_1": "Feature 2", 
                0: 'Correlation Coefficient'}
    
    # Every 2nd value is the reverse of the previous
    corr.drop(corr.iloc[1::2].index, inplace=True)
    corr.rename(columns=cols_map, inplace=True)
    
    return corr.drop(corr[corr['Correlation Coefficient'] == 1.0].index)


def combine_df(df1, df2):
    """
    combine_df combines df1 and df2 into a new dataframe assuming same columns
    
    :param df1: dataframe that will be at the start/top of resultant dataframe
    :param df2: dataframe that will be at the end/bottom of resultant dataframe
    :return: combined dataframe composed of df1 and df2
    """
    
    return pd.concat([df1, df2], sort=True).reset_index(drop=True)


def divide_df(df, first_len):
    """
    divide_df divides df into two parts, splitting after first_len rows of df and
    dropping 'Survived' column from the second dataframe
    
    :param df: dataframe desired to be split
    :param first_len: integer representing the number of rows in the first dataframe
    :return: two dataframes split from df
    """
    
    # Slicing with loc includes start AND end
    return df.loc[:first_len - 1], df.loc[first_len:].drop(['Survived'], axis=1)


def drop_cols(cols):
    """
    drop_cols drops col from both the train and test data sets
    
    :param cols: list of strings that signifies which column(s) to drop
    :return: None, columns dropped in place
    """
    for col in cols:
        train.drop([col], inplace=True, axis=1)
        test.drop([col], inplace=True, axis=1)
    
    return


def display_class_dist(percentages, y_label, title):
    """
    display_class_dist displays the distribution of classes among the 
    decks of the Titanic

    :param percentages: dictionary with passenger class proportions by deck, 
        returned by get_deck_class_count
    :param y_label: string label for y-axis of plot
    :param title: string title for plot
    :return: None
    """
    df_percent = pd.DataFrame(percentages).transpose()
    deck_names = percentages.keys()
    bar_count = np.arange(len(deck_names))
    bar_width = 0.75
    
    # Plot data
    plt.figure(figsize=(16, 8))
    plt.bar(bar_count, df_percent[0], 
            color='red', edgecolor='black', width=bar_width, 
            label='Passenger Class 1')
    plt.bar(bar_count, df_percent[1], bottom=df_percent[0], 
            color='lime', edgecolor='black', width=bar_width, 
            label='Passenger Class 2')
    plt.bar(bar_count, df_percent[2], bottom=df_percent[0] + df_percent[1], 
            color='blue', edgecolor='black', width=bar_width, 
            label='Passenger Class 3')
    
    # Tune plot
    plt.xlabel('Deck', size=25)
    plt.ylabel(y_label, size=25)
    plt.xticks(bar_count, deck_names)
    plt.tick_params(axis='x', labelsize=15)
    plt.tick_params(axis='y', labelsize=15)
    plt.legend(loc='upper right', prop={'size': 15})
    plt.title(title, size=30, y=1)   
    plt.show()
    
    return


def group_survivors(df, group, new_feature_name):
    """
    group_survivors calculates and creates a new feature denoting whether
    passengers in their given group are known to have survived or not using
    placeholder values in the new feature
    
    :param df: dataframe which has at least the same columns as the default 
        titanic dataframes
    :param group: list of strings denoting the features to be used for grouping
    :param new_feature_name: string of the new feature to be calculated
    :return: updated version of df with the new feature included
    """
    df[new_feature_name] = DEFAULT_SURVIVAL
    
    for _, group_df in df.groupby(group):
        if len(group_df) > 1:
            # Store whether any group members are known to have survived/died
            surv_max = group_df['Survived'].max()
            surv_min = group_df['Survived'].min()

            # Skip if no known survivors/deaths in training set
            if isnan(surv_max) and isnan(surv_min):
                continue

            # Assign non-default values to the new feature where
            # a member is known to have survived/died
            for _, row in group_df.iterrows():
                passId = row['PassengerId']
                if (surv_max == 1.0):
                    df.loc[df['PassengerId'] == passId, new_feature_name] = 1.0
                elif (surv_min==0.0):
                    df.loc[df['PassengerId'] == passId, new_feature_name] = 0.0
                    
    return df

In [ ]:
# Read the data
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
combined = combine_df(train, test)

**Understand the Dataset:**

To begin, we load in the data and then familiarize ourselves with the coarse grain aspects like datatypes, null values, etc.

Numerical:

- Training set has 891 samples
- Testing set has 418 samples
- Both *Age* and *Fare* features have distributions biased towards lower values with <1% high outliers
- 76% of passengers did not travel with parents or children
- 30% of the passengers had siblings and/or spouse aboard
- 38% of passengers survived in the training set compared to the actual survival rate of 32%

These can be seen by modifying *percentiles* in the following cell.

Categorical:

- 77% of *Cabin* values are missing, and of the ones that exist, 37% are duplicates
- There are 3 places to embark from, with 70% embarking from S=Southampton
- All but two values for *Name* are unique
- Male and Female options for *Sex*, with 64% of passengers being male
- 29% of *Ticket* values are duplicates and there is at least one ticket that shows up 11 times

In [ ]:
# Review 'Parch' distribution with 'percentiles=[.76, .77]'
# Review 'SibSp' distribution with 'percentiles=[.68, .69]'
# Review 'Age' and 'Fare' distribution with 'percentiles=[.9, .99]'
# Review 'Survived' distribution with 'percentiles=[.61, .62]'
percentiles = [.25, .50, .75]

# View the data
display(combined.head())
display(combined.describe(percentiles=percentiles))
display(combined.describe(include=['O']))
display(combined.info())

# Remove rows with missing target
train.dropna(axis=0, subset=['Survived'], inplace=True)

# Check which columns have null values and print
print('Number of null values by feature in combined train and test set:', 
      combined.isnull().sum()[combined.isnull().sum()>0], sep='\n')

**Investigate Assumptions:**

It is worth noting some assumptions based on the information we have.  We know that women and children had priority when evacuating.  Consequently, we expect women to be overrepresented in *Survived*.  We also know that class always comes into play when resources are limited (especially since first class accomodations were closest to lifeboats).  By the same logic, we expect passengers in class 1 will be more likely to survive than their class 2 and 3 counterparts.  We will test these assumptions noting that we have not yet filled in all *Age* values, so our results will not be completely accurate.

In [ ]:
train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

We can see above that there is a very significant disparity in survival rates between men and women.  Next we can check survival rate differences between adults and children.

In [ ]:
adults = train[train['Age'] >= CHILD_AGE_END]
children = train[train['Age'] < CHILD_AGE_END]

print('Proportion of passengers <{} who survived: {:.4f}'.format(CHILD_AGE_END, children['Survived'].mean()))
print('Proportion of passengers >={} who survived: {:.4f}'.format(CHILD_AGE_END, adults['Survived'].mean()))

There is not as large of a disparity in this case, but it is still significant.  Finally we take a look at the survival rates by passenger class.

In [ ]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Again, we see that there is a significant difference in survival rates based on this feature.  Our assumptions were well-founded.  We will proceed with these results in mind as we process the data and make predictions.

**Filling Fare Null Values:**

There is only one null value for *Fare*.  *Pclass* is certainly relevant for *Fare*.  It seems like having no family members travelling with them would also affect *Fare* (*Parch* = 0 and *SibSp* = 0).  Knowing this, we can fill in the median value of similar passengers.

In [ ]:
combined[combined['Fare'].isnull()]

We can see below that the median fare increases not only as the class of the ticket increases, but also as the number of family members travelling with the individual increases (there are exceptions, but the trend is there).

In [ ]:
# Calculate median fares by class and number of family members who are also passengers (training data)
med_fare = train.groupby(['Pclass', 'Parch', 'SibSp']).Fare.median()
print('Median fares for passengers by Pclass, Parch, and SibSp combinations:', 
      med_fare, sep='\n')

Seeing this trend, we can now confidently fill our null values using our proposed method.  We ensure that the calculation of parameters is done exclusively using the training set to avoid leakage.

In [ ]:
# Filling the missing value in Fare with the median Fare of 3rd class alone passenger
combined['Fare'] = combined['Fare'].fillna(med_fare[3][0][0])

**Filling Embarked Null Values:**

The only null values for *Embarked* are from two first class women who have the same ticket number.  From this, we will proceed assuming that they boarded the ship from the same port (one of C = Cherbourg, Q = Queenstown, or S = Southampton).

In [ ]:
combined[combined['Embarked'].isnull()]

Since the story of the Titanic and its sinking is so well researched, just Googling their names reveal the true value of our missing values.  [Encylopedia Titanica](https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html) tells us that

> Mrs Stone boarded the Titanic in Southampton on 10 April 1912 and was travelling in first class with her maid Amelie Icard. She occupied cabin B-28.

With this, we fill in the Embarked missing values with S.

In [ ]:
combined['Embarked'] = combined['Embarked'].fillna('S')

**Filling Age Null Values:**

We first check to see if *Age* is correlated with other numeric features.   If it is correlated with other features, we can fill in our missing values more appropriately than a simple global average.

In [ ]:
# Get pairwise correlations of 'Age' feature with the other numeric features
train, test = divide_df(combined, TRAIN_LEN)
corr_train = get_corr(train)
corr_train[(corr_train['Feature 1'] == 'Age') | 
           (corr_train['Feature 2'] == 'Age')]

We see that *Age* is correlated with *Pclass*.  *Sex* together with *Pclass* provides yet better predictions seen by the varied medians below.  We see two distinct trends, women having a lower median age than men for a given class, as well as median age increasing as class increases (class 1 is 'higher' than class 3 ie. more expensive).  As a result, it is necessary to take these relations into account when filling null values.

In [ ]:
# Calculate median ages for each Pclass and sex combinations (training data)
age_pclass_sex = train.groupby(['Pclass', 'Sex']).median()['Age']

# Print data for easier viewing
print('Median ages for the following groups (training data):')
for pclass in range(1, train['Pclass'].nunique() + 1):
    for sex in ['female', 'male']:
        print('Pclass {} {}s: {}'.format(pclass, sex, age_pclass_sex[pclass][sex]))
print('All passengers: {}'.format(train['Age'].median()))

We will now take this information and use it to fill the null values for *Age* using the median of their class and sex combinations (possible since no null values for either *Pclass* or *Sex*).

In [ ]:
# Filling the missing values in Age with the medians of Sex and Pclass groups
combined['Age'] = combined.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.median()))

**Create Deck Feature:**

*Cabin* is the feature with the most null values, a total of 879/1130 between the train and test sets.  This is too large of a proportion to impute, and the data would likely not be very valuable even if we did.  Instead, we will create a new feature, *Deck*, using the first letter of the *Cabin* values which indicates the deck where the cabin is.  We will denote all of the null values for *Cabin* with 'N' for *Deck*.

In [ ]:
# Creating Deck column from the first letter of the Cabin column (N denotes a null entry)
combined['Deck'] = combined['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'N')
train, test = divide_df(combined, TRAIN_LEN)

# Create counts for all Deck, Pclass combinations (training data)
deck_class_count = train.groupby(['Deck', 'Pclass']).count().rename(columns={'Name': 'Count'})
deck_class_count = deck_class_count[['Count']]
print('Passenger counts for each Deck, Pclass combination (training data) where N deck denotes null values:', 
      deck_class_count, sep='\n')

The previous cell's output provides some useful insight into how the decks are laid out.  However, since the number of cabins per deck is not constant, a proportion is more illustrative.  The following cell calculates the proportion of each passanger class by deck (training data) and then plots it.

In [ ]:
# Calculate count and proportion of passenger 'Deck', 'Pclass' combinations
deck_class_count, deck_class_prop = get_deck_class_count(deck_class_count, True)

# Plot Passenger class distribution by deck
y_label = 'Passenger Class Proportion (%)'
title = 'Passenger Class Distribution by Deck\n for Titanic Final Voyage (Training Set)'
display_class_dist(deck_class_prop, y_label, title)

The 'T' deck shown above is the boat deck (top deck) and only has 1 passenger.  For reference, decks 'A' through 'G' start just below the boat deck and run down towards the bottom of the ship.

In [ ]:
display(combined[combined['Deck'] == 'T'])
print('Median fare for similar passengers (first class with no family onboard): {:.2f}'
      .format(med_fare[1][0][0]))

We will move them to 'A' deck, the closest physically to the boat deck which conveniently only has other first class passengers.  This will ideally lead to better predictions as we only have a samply size of one for 'T' deck.  We no longer need the *Cabin* feature either now that we have extracted the data we could from it.

In [ ]:
# Passenger in the T deck is changed to A
combined = combine_df(train, test)
i = combined[combined['Deck'] == 'T'].index
combined.loc[i, 'Deck'] = 'A'

deck_class_count.pop('T', None)
deck_class_count['A'][1] += 1

# Drop the Cabin feature
COLS_TO_DROP.extend(['Cabin'])

**Feature Correlations:**

Now that we have concluded dealing with the null values, we can investigate the correlations between our features to look for further insights into the dataset.  These insights will guide us in engineering other features as well as illuminate any possibly redundant features.

In [ ]:
# Get training set correlations, focus on high correlations
train, test = divide_df(combined, TRAIN_LEN)
corr_train = get_corr(train)
high_corr_train = corr_train['Correlation Coefficient'] > 0.1
print('Training set correlations (coefficient > 0.1):')
display(corr_train[high_corr_train])

# Get testing set correlations, focus on high correlations
corr_test = get_corr(test)
high_corr_test = corr_test['Correlation Coefficient'] > 0.1
print('Testing set correlations (coefficient > 0.1):')
display(corr_test[high_corr_test])

**Creating DeckClassSurvProp feature:**

When investigating our initial assumptions, we saw that the survival rates for the various classes were quite disparate.  Later, when filling the null values for the *Cabin* feature, we saw that the various decks seemed to be organized in a way as to segregate the passenger classes.  These factors suggest that we may extract some predictive value from taking a closer look at the deck and class combinations, and their survival rates.

In [ ]:
# Create count for number of passengers survived based on 'Deck', 'Pclass' combinations
deck_class_surv_count = train[['Deck', 'Pclass', 'Survived']]
deck_class_surv_count = deck_class_surv_count.groupby(['Deck', 'Pclass']).sum()
deck_class_surv_count, _ = get_deck_class_count(deck_class_surv_count, False)

# Create proportion of passengers survived based on 'Deck', 'Pclass' combinations
deck_class_surv_prop = deck_class_surv_count.copy()
for deck in DECKS:
    for pclass in deck_class_count[deck].keys():
        try:
            deck_class_surv_prop[deck][pclass] = round((deck_class_surv_count[deck][pclass] / 
                                                       deck_class_count[deck][pclass]), 2)
        except ZeroDivisionError:
            pass

# Display information
print('Decimal percent of passengers survived for each \'Deck\', \'Pclass\' combination (training set)\':')
display(deck_class_surv_prop)

Comparing the training set and the combined set, we find that there are no *Deck*, *Pclass* combinations in the combined set that aren't already represented in the training set.  With that in mind, we can now safely apply these values as a new feature for both the train and test sets.  This feature is called *DeckClassSurvProp* and represents the proportion of passengers, with the same *Deck* and *Pclass* combination as the given passenger, who survived (only using data from training set to avoid leakage).

In [ ]:
# Apply new feature to our data sets
combined = combine_df(train, test)
combined['DeckPclassSurvProp'] = combined.apply(lambda x: get_surv_prop(x['Deck'], x['Pclass']),axis=1)

**Create FamilySize Feature:**

The features *SibSp* and *Parch* are highly correlated and do not seem to independently provide valuable information.  As a result, we can improve our model by combining these two features into a new *FamilySize* feature which denotes the number of family members the passenger is travelling with, as well as themself.

In [ ]:
# Create new feature 'FamilySize' from 'SibSp' and 'Parch'and drop those features
combined['FamilySize'] = combined['SibSp'] + combined['Parch'] + 1
COLS_TO_DROP.extend(['SibSp', 'Parch'])
train, test = divide_df(combined, TRAIN_LEN)

Now we plot counts of *FamilySize* as well as checking the *Survived* distribution for each *FamilySize*.

In [ ]:
# Plot relevant data
fig, axs = plt.subplots(figsize=(20, 20), ncols=2)
sns.barplot(x=train['FamilySize'].value_counts().index, 
            y=train['FamilySize'].value_counts().values, ax=axs[0])
sns.countplot(x='FamilySize', hue='Survived', data=train, ax=axs[1])

# Tune plot
axs[0].set_xlabel('Family Size')
axs[0].set_ylabel('Count')
axs[0].set_title('Family Size Value Counts', size=20)
axs[1].set_xlabel('Family Size')
axs[1].set_ylabel('Count')
axs[1].set_title('Survived Counts by Family Size', size=20)
axs[1].legend(['Died', 'Survived'], loc='upper right', prop={'size': 15})
plt.subplots_adjust(top=0.4)
plt.show()

**Create FamilySizeCat Feature:**

We can see in the 'Survived Counts by Family Size' plot that there seems to be significant trends in survival based on *FamilySize*.  Those who are alone have comparably low chances of survival compared to small families of 2-4.  For families of size >=5, we see that people are again less likely to survive than die.  As a result, we will group the values of *FamilySize* together into the following categories and create a new feature called *FamilySizeCat*.

- 1: Alone
- 2-4: Small
- 5-11: Large

In [ ]:
# Create mapping of 'FamilySize' to its corresponding category and create new feature
familysize_map = {1: 'Alone', 2: 'Small', 3: 'Small', 4: 'Small', 5: 'Large', 
                  6: 'Large', 7: 'Large', 8: 'Large', 11: 'Large'}
combined = combine_df(train, test)
combined['FamilySizeCat'] = combined['FamilySize'].map(familysize_map)
train, test = divide_df(combined, TRAIN_LEN)

# Plot relevant data
fig, axs = plt.subplots(figsize=(20, 20), ncols=2)
sns.barplot(x=train['FamilySizeCat'].value_counts().index, 
            y=train['FamilySizeCat'].value_counts().values, ax=axs[0])
sns.countplot(x='FamilySizeCat', hue='Survived', data=train, ax=axs[1])

# Tune plot
xticks = ['Alone', 'Small', 'Large']
axs[0].set_xlabel('Family Size Category')
axs[0].set_ylabel('Count')
axs[0].set_title('Family Size Category Value Counts', size=20)
axs[1].set_xlabel('Family Size Category')
axs[1].set_ylabel('Count')
axs[1].set_title('Survived Counts by Family Size Category', size=20)
axs[1].legend(['Died', 'Survived'], loc='upper right', prop={'size': 15})
plt.subplots_adjust(top=0.4)
plt.show()

**Create Title Feature:**

The *Name* feature appears to have titles ('Mr', 'Mrs', 'Master', etc.) for all passengers aboard.  We will take this part from *Name* and make it its own feature.

In [ ]:
# Create new feature to extract passenger's title from the Name column
combined = combine_df(train, test)
combined['Title'] = combined['Name'].apply(lambda name: name.split(',')[1].split('.')[0].strip())

# View the various values of these titles
print('Count of passenger titles aboard the Titanic:')
combined['Title'].value_counts()

We can see that almost all the titles have too small of a sample size for the model to extract information from.  We will use the strategy that Peter Begle used in an article on [Medium](https://medium.com/i-like-big-data-and-i-cannot-lie/how-i-scored-in-the-top-9-of-kaggles-titanic-machine-learning-challenge-243b5f45c8e9) to normalize these titles and give our model something to work with.

In [ ]:
# Normalize the titles
normalized_titles = {
    "Capt":         "Officer",
    "Col":          "Officer",
    "Don":          "Royalty",
    "Dona":         "Royalty",
    "Dr":           "Officer",
    "Jonkheer":     "Royalty",
    "Lady" :        "Royalty",
    "Major":        "Officer",
    "Master" :      "Master",
    "Miss" :        "Miss",
    "Mlle":         "Miss",
    "Mme":          "Mrs",
    "Mr" :          "Mr",
    "Mrs" :         "Mrs",
    "Ms":           "Mrs",
    "Rev":          "Officer",
    "Sir" :         "Royalty",
    "the Countess": "Royalty"}

# Map the current titles to the normalized titles
combined['Title'] = combined['Title'].map(normalized_titles)

# View the new normalized values of these titles
print('Count of updated passenger titles aboard the Titanic:')
combined['Title'].value_counts()

Now that they are organized succinctly, we check to see the survival rates by title.

In [ ]:
print('Decimal percentages for survival based on passenger title:')
combined[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()\
                               .sort_values(by='Survived', kind="quicksort", ascending=False)

We can see that title provides us with useful information as the likelihood of survival is drastically different between titles.  As a result, we will later use ordinal encoding to encapsulate the inherent hierarchy in likelihood of survival.

**Create Surname Feature:**

Another feature we can create is a *Surname* feature from the *Name* feature.  This may be useful as it seems likely that if one family member survived, others or perhaps all of the family would have survived.

In [ ]:
combined['Surname'] = combined['Name'].apply(lambda x: str.split(x, ",")[0])

However, it is important to take *Surname* feature with a grain of salt as different families travelling onboard can have the same *Surname*.  We can see that with the 'Davies' *Surname*.  There appears to be some strange data for these individuals that is incompatible.  Namely the *Parch*, *SibSp*, and *Ticket* values.  

In [ ]:
display(combined.loc[combined['Surname'] == 'Davies'])

**Create FamilySurvival Feature:**

Due to the issue with the *Surname* feature mentioned above, we will need to minimize the issue of grouping together passengers who are not travelling together.  This will be done by grouping passengers together based on *Surname*, *FamilySize*, and *Ticket*.  *Ticket* is included due to the observation that the *Ticket* is shared across all passengers travelling together.  We can see below from the largest family travelling on the Titanic.

In [ ]:
combined.loc[combined['FamilySize'] == 11]

Given this fact, we will use it to group together families aboard the Titanic.  Note that the value for *DEFAULT_SURVIVAL* of 0.5 is just a placeholder and will be replaced during encoding.  The value is just to differentiate between families with known survivors and known deaths.  The idea here is that passengers in families with known survivors are more likely to survive.  Contrarily, passengers in families with known deaths are less likely to survive.

In [ ]:
combined = group_survivors(combined, ['Surname', 'Fare', 'FamilySize'], 'FamilySurvival')
print('Count of passengers with family survival data: ', 
      combined.loc[combined['FamilySurvival']!=0.5].shape[0])

**Create GroupSurvival Feature:**

Similar to the reasoning above for the *FamilySurvival* feature, we will create a new feature *GroupSurvival* simply based on having the same *Ticket*.  There will be overlap between the two features, but there is still some usefulness to be extracted.  Passengers who are family would be likely to stick together and as a result, likely survive or perish together.  The sentiment can be extended to passengers who are travelling together but are not family (ie. friends, nannies, etc.).  These individuals may not be in the same cabin as would be true for families, so sticking together would not be as easy during the chaos of the sinking ship.

In [ ]:
combined = group_survivors(combined, ['Ticket'], 'GroupSurvival')
print('Count of passenger with group survival data: ', 
      combined[combined['GroupSurvival']!=0.5].shape[0])

**Discretize Continuous Features:**

Now we can discretize our continuous features (*Age*, *Fare*) by placing the data into bins where each bin has an equal ($\pm 1$) number of passengers.  This provides a better signal-to-noise ratio so long as the bins are not too large.  This binning also serves to encode our *Age* and *Fare* features as the *labels* parameter to *qcut* is provided.

In [ ]:
# Binning 'Age' feature
AGE_CUTS = 10
train, test = divide_df(combined, TRAIN_LEN)
train['Age'], age_bins = pd.qcut(train['Age'], AGE_CUTS, labels=list(range(AGE_CUTS)), retbins=True)
age_bins[0] = -1
test['Age'] = pd.cut(test['Age'], labels=list(range(AGE_CUTS)), bins=age_bins)

# Binning 'Fare' feature
FARE_CUTS = 10
train['Fare'], fare_bins = pd.qcut(train['Fare'], FARE_CUTS, labels=list(range(FARE_CUTS)), retbins=True)
fare_bins[0] = -1
test['Fare'] = pd.cut(test['Fare'], labels=list(range(FARE_CUTS)), bins=fare_bins)

# Return 'Age' and 'Fare' features back to integers from category
combined = combine_df(train, test)
combined[['Age', 'Fare']] = combined[['Age', 'Fare']].astype('int32')

Now we look at our binned groups for *Age*.  Keep in mind that the x-axis labels are the top end of the age range, so the count includes all values from the previous bar label to the current one ($[0, 16)$, $[16, 20)$, etc.).

In [ ]:
fig, axs = plt.subplots(figsize=(22, 9))
sns.countplot(x='Age', hue='Survived', data=train)

plt.xlabel('Age (range upper boundaries)')
plt.ylabel('Count')
plt.xticks(ticks=list(range(10)), labels=age_bins[1:])
plt.legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 15})
plt.title('Survival Counts in {} Feature'.format('Age'), size=15)
plt.show()

We see that survival rates are quite different between the various age bins with only the $[0, 16)$ and $[34, 40)$ age groups with a survival rate $>50\%$.  The same plot as above for *Age* is now provided for *Fare*.  Keep in mind again that the x-axis labels are the upper boundary for the age range bin ($[0, 7.55)$, $[7.55, 7.85)$, etc.).

In [ ]:
# Round fares to cents
fare_bins = [round(fare, 2) for fare in fare_bins]

# Plot Fare bins
fig, axs = plt.subplots(figsize=(22, 9))
sns.countplot(x='Fare', hue='Survived', data=train)

# Tune plot
plt.xlabel('Fare (range upper boundaries)')
plt.ylabel('Count')
plt.xticks(ticks=list(range(10)), labels=fare_bins[1:])
plt.legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 15})
plt.title('Survival Counts in {} Feature'.format('Fare'), size=15)
plt.show()

We can see a much clearer trend for *Fare* than we could for *Age*.  The survival rate consistently trends upwards as *Fare* increases (with the exception of the outlier $[27.00, 39.69)$ range).  

The number of bins for discretization of *Age* and *Fare* is not too large as the bins are able to uniquely capture the various behaviour of how these features affect *Survived*.

**Encode Features:**

Now we encode our desired features.  We will begin using ordinal encoding for the *Deck* and *Title* features.

In [ ]:
# Define the ordering of our encoding on our desired features
ord_cols = ['Deck', 'Title', 'FamilySurvival', 'GroupSurvival']
deck_cat = ['N', 'G', 'F', 'E', 'D', 'C', 'B', 'A']
title_cat = ['Mr', 'Officer', 'Master', 'Royalty', 'Miss', 'Mrs']
family_cat = group_cat = [0.0, 0.5, 1.0]

# Encode features
ord_enc = OrdinalEncoder(categories=[deck_cat, title_cat, family_cat, group_cat])
combined[ord_cols] = ord_enc.fit_transform(combined[ord_cols])

Next, we use one hot encoding on our remaining categorical variables.

In [ ]:
# OneHotEncode our desired features
oh_cols = ['Embarked', 'FamilySizeCat', 'Sex']
combined = pd.get_dummies(data=combined, columns=oh_cols)
train, test = divide_df(combined, TRAIN_LEN)

**Remove Unnecessary Features:**

Finally, we can remove any features that have become redundant, or simply not helpful for our purposes.

In [ ]:
# Simplify dataset, but keep ids for submission
passengerid_test = test.PassengerId
COLS_TO_DROP.extend(['PassengerId', 'Name', 'Ticket', 'Surname'])
drop_cols(COLS_TO_DROP)

# View data after all preprocessing
display(train.head())

**Create Model:**

Now that we are finished with preprocessing the data, we can begin to initialize our model.  We begin by defining our training, validation, and testing features/targets.

In [ ]:
# Explicitly split target from features
y = train[['Survived']].copy()
train.drop(columns='Survived', inplace=True)

# Break off validation set from training data, separate target from features
X_train, X_valid, y_train, y_valid = train_test_split(train, y, train_size=0.8, random_state=RNG_SEED)

# Rename for variable name consistency
X_test = test.copy()

Then we define our model and use the best parameters we are able to find using *GricSearchCV* on our training data.  Change *WANT_HYPERPARAMETERS = TRUE* if the calculation is desired.  *WANT_HYPERPARAMETERS = False* will give best parameters from previous runs for the sake of time.

In [ ]:
# Keep False to use best parameters from previous run
WANT_HYPERPARAMETERS = False

if WANT_HYPERPARAMETERS:
    # Define parameters ranges to be tested
    params = dict(max_depth = [n for n in range(3, 9)], 
                  min_samples_split = [n for n in range(2, 4)], 
                  min_samples_leaf = [n for n in range(2, 4)],
                  n_estimators = [20, 40, 60, 80],)

    # Define the model and find best parameters
    model = GridSearchCV(RandomForestClassifier(random_state=RNG_SEED),
                         params, cv=5, scoring='accuracy')
    model.fit(X_train, y_train)

    print(f'Best parameters {model_random_forest.best_params_}')
    print(f'Mean cross-validated accuracy of the best parameters: {model.best_score_:.4f}')
    
else:
    model = RandomForestClassifier(n_estimators=50, max_depth=5, 
                                   min_samples_leaf=2, min_samples_split=2, 
                                   random_state=RNG_SEED)
    model.fit(X_train, y_train)

Now that the model is initialized and trained with the best parameters we could find, we use cross validation on different validation sets to get an accurate view of the model's performance on our training set.

In [ ]:
# Processing of validation data, get predictions
predictions_valid = model.predict(X_valid)

# Evaluate the model using validation set
score = mean_absolute_error(y_valid, predictions_valid)
print('MAE for validation set prediction:', round(score, 4))

# Multiply by -1 since sklearn calculates negative MAE
folds = 5
scores = -1 * cross_val_score(model, X_valid, y_valid, cv=folds, scoring='neg_mean_absolute_error')

print('Average MAE score (across experiments using {} folds):'.format(folds))
print(round(scores.mean(), 4))

**Feature Importance:**

Now that the model is trained and cross-validated, we check to see the feature importances.

In [ ]:
# Sort values but keep indices
sorted_idx = model.feature_importances_.argsort()

# Plot data
plt.figure(figsize=(15, 15))
plt.barh(X_test.columns[sorted_idx], model.feature_importances_[sorted_idx])
plt.xlabel('Feature Importance Coefficient')
plt.ylabel('Feature')
plt.title('Random Forest Classifier Feature Importances')
plt.show()

**Submission:**

Finally, we calculate and submit our final predictions.

In [ ]:
# Processing of test data, get predictions
predictions_test = model.predict(X_test)
predictions_test = predictions_test.astype(int)

# Save test predictions to file
output = pd.DataFrame({'PassengerId': passengerid_test,
                       'Survived': predictions_test})
output.to_csv('submission.csv', index=False)

Thank you to [Gunes Etivan's Titanic - Advanced Feature Engineering Tutorial](http://https://www.kaggle.com/gunesevitan/titanic-advanced-feature-engineering-tutorial/notebook) and [Manav Sehgal's Titanic Data Science Solutions](http://https://www.kaggle.com/startupsci/titanic-data-science-solutions/notebook) for their helpful ideas and overall contribution to this competition.